In [7]:
import pandas as pd
import numpy as np
import urllib.request
import codecs
from bs4 import BeautifulSoup

In [8]:
edgar = pd.read_csv('edgar.csv').values
edgar_link = 'https://www.sec.gov/Archives/'

In [5]:
def clean_html(saved_name, file_url):
    with urllib.request.urlopen(file_url) as response:
        html_file = response.read()
    soup = BeautifulSoup(html_file, 'lxml').get_text()
    temp_file = codecs.open('data/' + saved_name + '.txt', 'w', 'utf-8')
    temp_file.write(soup)
    temp_file.close()

In [6]:
for i in range(50):
    if edgar[i][2] == '10-Q':
        clean_html(edgar[i][3] + '-' + str(edgar[i][1]), edgar_link + edgar[i][4])

9590


9921


False
